In [3]:
# Turn off unnecessary warnings
import warnings
warnings.filterwarnings("ignore")

# Import all the required packages
import json
import nltk
import urllib
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.wsd import lesk
from nltk.parse import CoreNLPParser
from nltk.parse.corenlp import CoreNLPDependencyParser


# Performs Word tokenization on sentences
def Tokenization(sentence):
    tokens = nltk.word_tokenize(sentence)
    return tokens


# Performs Word Lemmatization
def Lemmatization(word_tokens):
    lemmas = []
    for token in word_tokens:
        lemmas.append(wordnet_lemmatizer.lemmatize(token))
    return lemmas


# Performs POS tagging
def POSTagging(word_tokens):
    POStags = nltk.pos_tag(word_tokens)
    return POStags   


# Obtains sentence heads
def getHeads(sentence, word_tokens):
    # Set up dependency parser
    dependencyParser = CoreNLPDependencyParser(url='http://localhost:9000')
    headList = []
    
    # Split the sentence
    stripedSen = sentence.strip(" '\"")
    if stripedSen != "":
        # Perform dependency parse
        depParse = dependencyParser.raw_parse(stripedSen)
        parseTree = list(depParse)[0]
        headWord = ""
        headWord = [k["word"] for k in parseTree.nodes.values() if k["head"] == 0][0]
        
        # Appends head if it's not empty
        if headWord != "":
            headList.append([headWord])
            
        # Obtain head word based on two cases
        else:
            for i, pp in enumerate(tagged):
                if pp.startswith("VB"):
                    headList.append([word_tokens[i]])
                    break
            if headWord == "":
                for i, pp in enumerate(tagged):
                    if pp.startswith("NN"):
                        headList.append([word_tokens[i]])
                        break
                        
    # For empty sentence, we just append "" as head
    else:
        headList.append([""])
 
    return headList


# Obtains WordNet Features
def WordNetFeatures(sentence, word_tokens):
    # Creates dictionaries for important word senses
    hypernyms_list = []
    hyponyms_list = []
    meronyms_list = []
    holonyms_list = []
    synonyms_list = []
    
    # Populates the above dictionaries according to the word senses associated with them
    for token in word_tokens:
        # Extracts best sense for each word using LESK
        best_sense = lesk(sentence, token)
        
        if best_sense is not None:
            # Obtains Synonyms
            synonym = token
            if best_sense. lemmas()[0].name() != token:
                synonym = best_sense.lemmas()[0].name()
            synonyms_list.append(synonym)
            
            # Obtains Hypernyms
            if best_sense.hypernyms() != []:
                hypernyms_list.append(best_sense.hypernyms()[0].lemmas()[0].name())
        
            # Obtains Hyponyms
            if best_sense.hyponyms() != []:
                hyponyms_list.append(best_sense.hyponyms()[0].lemmas()[0].name())
            
            # Obtains Meronyms
            if best_sense.part_meronyms() != []:
                meronyms_list.append(best_sense.part_meronyms()[0].lemmas()[0].name())
                
            # Obtains Holonyms
            if best_sense.part_holonyms() != []:
                holonyms_list.append(best_sense.part_holonyms()[0].lemmas()[0].name())
          
        # When there's no best sense, the token itself is the Synonym
        else:
            synonyms_list.append(token)
            
    return hypernyms_list, hyponyms_list, meronyms_list, holonyms_list, synonyms_list
   
    
# Performs Dependency Parsing
def DependencyParsing(sentence):
    dependencyParser = CoreNLPDependencyParser(url='http://localhost:9000')
    parse, = dependencyParser.raw_parse(sentence)
    
    # Dependency parsing to parse tree based patterns as features
    depParseResult = list(parse.triples())
    
    return depParseResult
    
    
# Main method
if __name__ == "__main__":
    # List of all article names in the repository
    articleNames = ["109.txt", "111.txt", "151.txt", "160.txt", "177.txt", 
                    "179.txt","181.txt", "196.txt", "199.txt", "220.txt", 
                    "222.txt", "226.txt", "247.txt", "273.txt", "281.txt", 
                    "282.txt", "285.txt", "287.txt", "288.txt", "297.txt", 
                    "304.txt", "342.txt", "347.txt", "360.txt", "390.txt", 
                    "400.txt", "428.txt", "56.txt", "58.txt", "6.txt"] 
    fileCount = len(articleNames)
    
    content = ""
    folderPath = "https://raw.githubusercontent.com/SaiManasaVedantam/NLP-QA-System-Datasets/main/Articles/"
    for i in range(fileCount):
        print("Started Processing File : " + articleNames[i])
        fileName = folderPath + articleNames[i]
        response = urllib.request.urlopen(fileName)
        webContents = response.read()
        stringTypeData = webContents.decode("utf-8")
        content = stringTypeData
        count = 0
        corpus_dict = {}

        # Obtain wordnet lemmatizer
        wordnet_lemmatizer = WordNetLemmatizer()

        # Get tokenized content
        sentences = []
        tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
        sentences.extend(tokenizer.tokenize(content))

        # Sentence count
        #print("Total Sentences After splitting the document: ", len(sentences))
        print("Extracting features for each sentence in the file...")
    
        # Extracting words
        for sen in sentences:
            print("\n------SENTENCE------")
            print(sen)

            word_tokens = Tokenization(sen)
            #print("\nWord Tokenization : Done")
            #print(word_tokens)

            word_lemmas = Lemmatization(word_tokens)
            #print("Word Lemmatization : Done")
            #print(word_lemmas)

            word_POStags = POSTagging(word_tokens)
            #print("POS Tagging : Done")
            #print(word_POStags)

            hypernyms, hyponyms, meronyms, holonyms, synonyms = WordNetFeatures(sen, word_tokens)
            #print("WordNet Feature Extraction : Done")
            #print(holonyms)
            
            depParse = DependencyParsing(sen)
            #print("Dependency Parsing : Done")
            #print(depParse)

            headList = getHeads(sen, word_tokens)
            #print("Obtaining Heads : Done")
            #print(headList)

            # Process data format to suit the Elastic Search requirements
            count = count + 1
            corpus_dict[count] = {}
            
            corpus_dict[count]["sentence"] = {}
            corpus_dict[count]["sentence"] = sen
            
            corpus_dict[count]["tokenized_text"] = {}
            corpus_dict[count]["tokenized_text"] = word_tokens
            
            corpus_dict[count]["lemma"] = {}
            corpus_dict[count]["lemma"] = word_lemmas
            
            corpus_dict[count]["tagged"] = {}
            corpus_dict[count]["tagged"] = word_POStags
            
            corpus_dict[count]["dependency_parse"] = {}
            corpus_dict[count]["dependency_parse"] = depParse
            
            corpus_dict[count]["synonyms"] = {}
            corpus_dict[count]["synonyms"] = synonyms
            
            corpus_dict[count]["hypernyms"] = {}
            corpus_dict[count]["hypernyms"] = hypernyms
            
            corpus_dict[count]["hyponyms"] = {}
            corpus_dict[count]["hyponyms"] = hyponyms
            
            corpus_dict[count]["meronyms"] = {}
            corpus_dict[count]["meronyms"] = meronyms
            
            corpus_dict[count]["holonyms"] = {}
            corpus_dict[count]["holonyms"] = holonyms
            
            corpus_dict[count]["head_word"] = {}
            corpus_dict[count]["head_word"] = headList[0]
            
            corpus_dict[count]["file_name"] = {}
            corpus_dict[count]["file_name"] = articleNames[i]

        output_name = '../Pipeline-Output/Parsed-' + articleNames[i]
        with open(output_name, 'w+', encoding='utf8') as output_file:
            json.dump(corpus_dict, output_file,  indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)
        
        print("Completed Processing File : " + articleNames[i])
        
    print("Task 1 Successfully Completed !!!")

Started Processing File : 109.txt
Extracting features for each sentence in the file...

------SENTENCE------
Bird migration is the regular seasonal movement, often north and south along a flyway, between breeding and wintering grounds.

------SENTENCE------
Many species of bird migrate.

------SENTENCE------
Migration carries high costs in predation and mortality, including from hunting by humans, and is driven primarily by availability of food.

------SENTENCE------
It occurs mainly in the northern hemisphere, where birds are funnelled on to specific routes by natural barriers such as the Mediterranean Sea or the Caribbean Sea.

------SENTENCE------
Historically, migration has been recorded as much as 3,000 years ago by Ancient Greek authors including Homer and Aristotle, and in the Book of Job, for species such as storks, turtle doves, and swallows.

------SENTENCE------
More recently, Johannes Leche began recording dates of arrivals of spring migrants in Finland in 1749, and scienti


------SENTENCE------
For larger birds, flying in flocks reduces the energy cost.

------SENTENCE------
Geese in a V-formation may conserve 12–20% of the energy they would need to fly alone.

------SENTENCE------
Red knots Calidris canutus and dunlins Calidris alpina were found in radar studies to fly 5 km/h (3.1 mph) faster in flocks than when they were flying alone.

------SENTENCE------
Birds fly at varying altitudes during migration.

------SENTENCE------
An expedition to Mt.

------SENTENCE------
Everest found skeletons of northern pintail Anas acuta and black-tailed godwit Limosa limosa at 5,000 m (16,000 ft) on the Khumbu Glacier.

------SENTENCE------
Bar-headed geese Anser indicus have been recorded by GPS flying at up to 6,540 metres (21,460 ft) while crossing the Himalayas, at the same time engaging in the highest rates of climb to altitude for any bird.

------SENTENCE------
Anecdotal reports of them flying much higher have yet to be corroborated with any direct evidence.




------SENTENCE------
They land in the morning and may feed for a few days before resuming their migration.

------SENTENCE------
The birds are referred to as passage migrants in the regions where they occur for short durations between the origin and destination.

------SENTENCE------
Nocturnal migrants minimize predation, avoid overheating, and can feed during the day.

------SENTENCE------
One cost of nocturnal migration is the loss of sleep.

------SENTENCE------
Migrants may be able to alter their quality of sleep to compensate for the loss.

------SENTENCE------
Many long-distance migrants appear to be genetically programmed to respond to changing day length.

------SENTENCE------
Species that move short distances, however, may not need such a timing mechanism, instead moving in response to local weather conditions.

------SENTENCE------
Thus mountain and moorland breeders, such as wallcreeper Tichodroma muraria and white-throated dipper Cinclus cinclus, may move only altitudinall


------SENTENCE------
Certain areas, because of their location, have become famous as watchpoints for such birds.

------SENTENCE------
Examples are the Point Pelee National Park in Canada, and Spurn in England.

------SENTENCE------
Reverse migration, where the genetic programming of young birds fails to work properly, can lead to rarities turning up as vagrants thousands of kilometres out of range.

------SENTENCE------
A related phenomenon called "abmigration" involves birds from one region joining similar birds from a different breeding region in the common winter grounds and then migrating back along with the new population.

------SENTENCE------
This is especially common in some waterfowl, which shift from one flyway to another.

------SENTENCE------
It has been possible to teach a migration route to a flock of birds, for example in re-introduction schemes.

------SENTENCE------
After a trial with Canada geese Branta canadensis, microlight aircraft were used in the US to teach sa


------SENTENCE------
The earliest written mention of immunity can be traced back to the plague of Athens in 430 BCE.

------SENTENCE------
Thucydides noted that people who had recovered from a previous bout of the disease could nurse the sick without contracting the illness a second time.

------SENTENCE------
Many other ancient societies have references to this phenomenon, but it was not until the 19th and 20th centuries before the concept developed into scientific theory.

------SENTENCE------
The study of the molecular and cellular components that comprise the immune system, including their function and interaction, is the central science of immunology.

------SENTENCE------
The immune system has been divided into a more primitive innate immune system and, in vertebrates, an acquired or adaptive immune system.

------SENTENCE------
The latter is further divided into humoral (or antibody) and cell-mediated components.

------SENTENCE------
The humoral (antibody) response is defined 

...


------SENTENCE------
A Tibetan translation in the 16th century preserves the letter of the Yongle Emperor, which the Association for Asian Studies notes is polite and complimentary towards the Karmapa.

------SENTENCE------
The letter of invitation reads, In order to seek out the Karmapa, the Yongle Emperor dispatched his eunuch Hou Xian and the Buddhist monk Zhi Guang (d. 1435) to Tibet.

------SENTENCE------
Traveling to Lhasa either through Qinghai or via the Silk Road to Khotan, Hou Xian and Zhi Guang did not return to Nanjing until 1407.

------SENTENCE------
During his travels beginning in 1403, Deshin Shekpa was induced by further exhortations by the Ming court to visit Nanjing by April 10, 1407.

------SENTENCE------
Norbu writes that the Yongle Emperor, following the tradition of Mongol emperors and their reverence for the Sakya lamas, showed an enormous amount of deference towards Deshin Shekpa.

------SENTENCE------
The Yongle Emperor came out of the palace in Nanjing to gr


------SENTENCE------
Silk workshops during the Ming also catered specifically to the Tibetan market with silk clothes and furnishings featuring Tibetan Buddhist iconography.

------SENTENCE------
While the Ming dynasty traded horses with Tibet, it upheld a policy of outlawing border markets in the north, which Laird sees as an effort to punish the Mongols for their raids and to "drive them from the frontiers of China."

------SENTENCE------
However, after Altan Khan (1507–1582)—leader of the Tümed Mongols who overthrew the Oirat Mongol confederation's hegemony over the steppes—made peace with the Ming dynasty in 1571, he persuaded the Ming to reopen their border markets in 1573.

------SENTENCE------
This provided the Chinese with a new supply of horses that the Mongols had in excess; it was also a relief to the Ming, since they were unable to stop the Mongols from periodic raiding.

------SENTENCE------
Laird says that despite the fact that later Mongols believed Altan forced the Min


------SENTENCE------
He writes that the Ming relationship with Tibet is problematic for China’s insistence of its unbroken sovereignty over Tibet since the 13th century.

------SENTENCE------
As for the Tibetan view that Tibet was never subject to the rule of the Yuan or Qing emperors of China, Sperling also discounts this by stating that Tibet was "subject to rules, laws and decisions made by the Yuan and Qing rulers" and that even Tibetans described themselves as subjects of these emperors.

------SENTENCE------
Josef Kolmaš, a sinologist, Tibetologist, and Professor of Oriental Studies at the Academy of Sciences of the Czech Republic, writes that it was during the Qing dynasty "that developments took place on the basis of which Tibet came to be considered an organic part of China, both practically and theoretically subject to the Chinese central government."

------SENTENCE------
Yet he states that this was a radical change in regards to all previous eras of Sino-Tibetan relations.


------SENTENCE------
Of the third Dalai Lama, China Daily states that the "Ming dynasty showed him special favor by allowing him to pay tribute."

------SENTENCE------
China Daily then says that Sonam Gyatso was granted the title Dorjichang or Vajradhara Dalai Lama in 1587 [sic!

------SENTENCE------
], but China Daily does not mention who granted him the title.

------SENTENCE------
Without mentioning the role of the Mongols, China Daily states that it was the successive Qing dynasty which established the title of Dalai Lama and his power in Tibet: "In 1653, the Qing emperor granted an honorific title to the fifth Dalai Lama and then did the same for the fifth Panchen Lama in 1713, officially establishing the titles of the Dalai Lama and the Panchen Erdeni, and their political and religious status in Tibet."

------SENTENCE------
Chen states that the fourth Dalai Lama Yonten Gyatso was granted the title "Master of Vajradhara" and an official seal by the Wanli Emperor in 1616.

------


------SENTENCE------
Van Praag states that Tibet and the Dalai Lama's power was recognized by the "Manchu Emperor, the Mongolian Khans and Princes, and the rulers of Ladakh, Nepal, India, Bhutan, and Sikkim."

------SENTENCE------
When the Dzungar Mongols attempted to spread their territory from what is now Xinjiang into Tibet, the Kangxi Emperor (r. 1661–1722) responded to Tibetan pleas for aid with his own expedition to Tibet, occupying Lhasa in 1720.

------SENTENCE------
By 1751, during the reign of the Qianlong Emperor (r. 1735–1796), a protectorate and permanent Qing dynasty garrison was established in Tibet.

------SENTENCE------
As of 1751, Albert Kolb writes that "Chinese claims to suzerainty over Tibet date from this time."
Completed Processing File : 6.txt
Task 1 Successfully Completed !!!
